In [1]:
import rqalpha as rqa
import rqalpha_plus
import rqdatac as rqd
import rqoptimizer as rqo
import datetime
import pickle
import os
import pandas as pd
pd.set_option('display.width',200)
rqd.init()

# 各行业各因子过去收益

# 各行业累计收益最大因子

In [12]:
industry_best_factor.apply(['idxmax','max'],axis=1)

,idxmax,max
农林牧渔,total_asset_turnover_ttm,4.652
采掘,ps_ratio_ttm,2.074
化工,inc_revenue_ttm,2.655
钢铁,pb_ratio_ttm,3.024
有色金属,profit_from_operation_to_revenue_ttm,2.563
电子,quick_ratio_ttm,3.301
家用电器,inventory_turnover_ttm,6.087
食品饮料,gross_profit_margin_ttm,10.029
纺织服装,pe_ratio_ttm,2.826
轻工制造,ps_ratio_ttm,2.931


---

# 因子数据

In [57]:
all_dates = total_results_df.index.levels[0]
dates = []
lastdate = None
for i in range(len(all_dates)):
    date = all_dates[i]
    if lastdate is None:
        lastdate = date
        continue
    if date.month != lastdate.month:
        dates.append(lastdate)
    lastdate = date

In [62]:
all_factor_data = {}
for date in dates:
    universe = rqd.all_instruments('CS',date)['order_book_id'].tolist()
    industry = rqd.shenwan_instrument_industry(universe, date).loc[:, ['index_name']]
    best_factor_all_industry = pd.Series()
    for i,j in industry.groupby(by='index_name'):
        # 表现最好的因子
        factor = total_results_series.loc[date,i]
        # 该行业股票列表
        stocklist = j.index.tolist()
        best_factor = rqd.get_factor(stocklist,factor,date,date)
        best_factor_all_industry = pd.concat([best_factor_all_industry,best_factor])
        best_factor_all_industry.dropna(inplace=True)
    all_factor_data[date] = best_factor_all_industry
    print(date,len(best_factor_all_industry))

F:\anaconda\envs\ricequant\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """


2015-03-31 00:00:00 2529
2015-04-30 00:00:00 2600
2015-05-29 00:00:00 2567
2015-06-30 00:00:00 2581
2015-07-31 00:00:00 2561
2015-08-31 00:00:00 2616
2015-09-30 00:00:00 2575
2015-10-30 00:00:00 2672
2015-11-30 00:00:00 2706
2015-12-31 00:00:00 2681
2016-01-29 00:00:00 2718
2016-02-29 00:00:00 2708
2016-03-31 00:00:00 2703
2016-04-29 00:00:00 2754
2016-05-31 00:00:00 2769
2016-06-30 00:00:00 2774
2016-07-29 00:00:00 2765
2016-08-31 00:00:00 2777
2016-09-30 00:00:00 2792
2016-10-31 00:00:00 2880
2016-11-30 00:00:00 2884
2016-12-30 00:00:00 2888
2017-01-26 00:00:00 2895
2017-02-28 00:00:00 2897
2017-03-31 00:00:00 2898
2017-04-28 00:00:00 3052
2017-05-31 00:00:00 3066
2017-06-30 00:00:00 3078
2017-07-31 00:00:00 3089
2017-08-31 00:00:00 3113
2017-09-29 00:00:00 3118
2017-10-31 00:00:00 3281
2017-11-30 00:00:00 3241
2017-12-29 00:00:00 3209
2018-01-31 00:00:00 3216
2018-02-28 00:00:00 3199
2018-03-30 00:00:00 3257
2018-04-27 00:00:00 3387
2018-05-31 00:00:00 3416
2018-06-29 00:00:00 3428


In [64]:
with open('factor/best_factor.pkl','wb') as pf:
    pickle.dump(all_factor_data,pf)